# Lambda School Data Science Module 123

## Introduction to Bayesian Inference

!['Detector! What would the Bayesian statistician say if I asked him whether the--' [roll] 'I AM A NEUTRINO DETECTOR, NOT A LABYRINTH GUARD. SERIOUSLY, DID YOUR BRAIN FALL OUT?' [roll] '... yes.'](https://imgs.xkcd.com/comics/frequentists_vs_bayesians.png)

*[XKCD 1132](https://www.xkcd.com/1132/)*


## Prepare - Bayes' Theorem and the Bayesian mindset

Bayes' theorem possesses a near-mythical quality - a bit of math that somehow magically evaluates a situation. But this mythicalness has more to do with its reputation and advanced applications than the actual core of it - deriving it is actually remarkably straightforward.

### The Law of Total Probability

By definition, the total probability of all outcomes (events) if some variable (event space) $A$ is 1. That is:

$$P(A) = \sum_n P(A_n) = 1$$

The law of total probability takes this further, considering two variables ($A$ and $B$) and relating their marginal probabilities (their likelihoods considered independently, without reference to one another) and their conditional probabilities (their likelihoods considered jointly). A marginal probability is simply notated as e.g. $P(A)$, while a conditional probability is notated $P(A|B)$, which reads "probability of $A$ *given* $B$".

The law of total probability states:

$$P(A) = \sum_n P(A | B_n) P(B_n)$$

In words - the total probability of $A$ is equal to the sum of the conditional probability of $A$ on any given event $B_n$ times the probability of that event $B_n$, and summed over all possible events in $B$.





### The Law of Conditional Probability

What's the probability of something conditioned on something else? To determine this we have to go back to set theory and think about the intersection of sets:

The formula for actual calculation:

$$P(A|B) = \frac{P(A \cap B)}{P(B)}$$

![Visualization of set intersection](https://upload.wikimedia.org/wikipedia/commons/9/99/Venn0001.svg)

Think of the overall rectangle as the whole probability space, $A$ as the left circle, $B$ as the right circle, and their intersection as the red area. Try to visualize the ratio being described in the above formula, and how it is different from just the $P(A)$ (not conditioned on $B$).

We can see how this relates back to the law of total probability - multiply both sides by $P(B)$ and you get $P(A|B)P(B) = P(A \cap B)$ - replaced back into the law of total probability we get $P(A) = \sum_n P(A \cap B_n)$.

This may not seem like an improvement at first, but try to relate it back to the above picture - if you think of sets as physical objects, we're saying that the total probability of $A$ given $B$ is all the little pieces of it intersected with $B$, added together. The conditional probability is then just that again, but divided by the probability of $B$ itself happening in the first place.


### Bayes Theorem

Here is is, the seemingly magic tool:

$$P(A|B) = \frac{P(B|A)P(A)}{P(B)}$$

In words - the probability of $A$ conditioned on $B$ is the probability of $B$ conditioned on $A$, times the probability of $A$ and divided by the probability of $B$. These unconditioned probabilities are referred to as "prior beliefs", and the conditioned probabilities as "updated."

Why is this important? Scroll back up to the XKCD example - the Bayesian statistician draws a less absurd conclusion because their prior belief in the likelihood that the sun will go nova is extremely low. So, even when updated based on evidence from a detector that is $35/36 = 0.972$ accurate, the prior belief doesn't shift enough to change their overall opinion.

### Using Bayes Theorem Iteratively (repeated testing)

This example comes from [Wikipedia](https://en.wikipedia.org/wiki/Bayes%27_theorem)

There are many ways to apply Bayes' theorem - one less absurd example is to apply it to drug tests. You may think that a drug test that is 100% accurate for true positives (detecting somebody who is a user) is pretty good, but what if it also has 1% false positive rate (indicating somebody is a user when they're not)? And furthermore, the rate of drug use in the population at large (and thus our prior belief) is 1/200.

What is the likelihood somebody really is drunk if they test positive? Some may guess it's 99% - the difference between the true positives and the false positives. But we have a prior belief of the background/true rate of drug use. Sounds like a job for Bayes' theorem!

![Bayes Theorem Drug Test Example](https://wikimedia.org/api/rest_v1/media/math/render/svg/95c6524a3736c43e4bae139713f3df2392e6eda9)

In other words, the likelihood that somebody is a user given they tested positive on a drug test is only 33.2% - probably much lower than you'd guess. This is why, in practice, it's important to use repeated testing to confirm. If we have the same individual who tested positive the first time take the drug test a second time then the posterior probability from our the first test becomes our new prior during the second application. What is the probability that a person is a drug user after two positive drug tests in a row?

Bayes' theorem has been relevant in court cases where proper consideration of evidence was important. Whether it's a drug test, breathalyzer, pregnancy test, doctor's diagnosis, or neutrino detector, we have to take into account **both** the false positive rate and our prior probability in order to calculate the correct conditional probability.

In [ ]:
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
 # 33.4% likelihood of truly having corona virus given that the test said you are positive

0.33444816053511706

In [ ]:
## subsequent application of the test
# True positive rate
p_pos_infected = 1
# Prior Probability = Come from the posterior of the previous test
p_infected = 0.33444816053511706
# False Positive Rate
p_pos_not_infected = .01
# compliment of the prior (1-event == compliment)
p_not_infected = 1 - p_infected

numerator = p_pos_infected * p_infected

denominator = (p_pos_infected * p_infected) + (p_pos_not_infected * p_not_infected)

posterior_probability = numerator/denominator

posterior_probability # 98% likelihood of truly having corona virus if tested twice and resulted positive

0.9804882831650161

## Live Lecture - Deriving Bayes' Theorem, Calculating Bayesian Confidence

Notice that $P(A|B)$ appears in the above laws - in Bayesian terms, this is the belief in $A$ updated for the evidence $B$. So all we need to do is solve for this term to derive Bayes' theorem. Let's do it together!

In [ ]:
# Activity 2 - Use SciPy to calculate Bayesian confidence intervals
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.bayes_mvs.html#scipy.stats.bayes_mvs

In [ ]:
coinflips = np.random.binomial(1, .5, 20)
coinflips

array([0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1])

In [ ]:
# frequentist confidence interval
def confidence_interval (data, confidence =0.95):
  data = np.array(data)
  mean = np.mean(data)
  n = len(data)
  # pass extra parameter (ddof=1) to calculate **sample** standard deviation
  s = data.std(ddof=1)
  stderr = s / np.sqrt(n)
  # Lookup the t-statistic that corresponds to 95% area under the curve
  # for my given degreen of freedom: 49
  t = stats.t.ppf((1 + confidence) / 2.0, n - 1)
  margin_of_error = t*stderr
  return (mean - margin_of_error, mean, mean + margin_of_error)

In [ ]:
confidence_interval(coinflips) # frequentist

(0.21111712307712144, 0.45, 0.6888828769228785)

In [ ]:
## Bayesian Confidence Interval
stats.bayes_mvs(coinflips)

(Mean(statistic=0.45, minmax=(0.2526490810136677, 0.6473509189863322)),
 Variance(statistic=0.29117647058823537, minmax=(0.16421435906388618, 0.4892748451302152)),
 Std_dev(statistic=0.5317348479252392, minmax=(0.4052337091900009, 0.6994818404577886)))

In [ ]:
# mean confidence interval
mean_CI, _, _ = stats.bayes_mvs(coinflips, alpha=.95) # bayesian is using a 90% confidence int by default
# added alpha (to get identical result with frequentist that used 95% confidence interval)
mean_CI

Mean(statistic=0.45, minmax=(0.21111712307712144, 0.6888828769228785))

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)